In [1]:
import pandas as pd
import numpy as np
from src.static import DATA_DIR

In [2]:
mapped_data = pd.read_csv(f'{DATA_DIR}/mapped_data.csv', na_values=['.','NaN', 'None'])
mapped_data.sort_values(by=['year', 'enterprise_flag', 'record_number'], inplace=True)

In [3]:
groups = ['year', 'enterprise_flag', 'record_number', '']
test = mapped_data.groupby(['year', 'enterprise_flag', 'record_number']).agg(set).reset_index()
test.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_bedrooms,num_units,affordability_level,tenant_income_ind
0,2008,fannie,1,{<10%},"{>10, <=120%}","{<20%, >=40%}",{prior to year aquired},{refinance},{mortgage_company},{no},{120},"{>=2, 0-1}","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 10.0, 11.0, 12....","{>50, <=60%, >=0, <=50%}",{No}
1,2008,fannie,2,{>=30% <100%},"{>0, <=80%}","{>=20%, <40%}",{prior to year aquired},{refinance},{mortgage_company},{no},{72},"{>=2, 0-1}","{1.0, 2.0, 4.0, 5.0, 8.0, 9.0, 24.0}","{>60, <=80%, >80, <=100%, >=0, <=50%}",{No}
2,2008,fannie,3,{<10%},{>120%},"{<20%, <40%}",{prior to year aquired},{purchase},{mortgage_company},{no},{264},"{>=2, 0-1}",{1.0},"{>50, <=60%, >60, <=80%, >=0, <=50%, >100% }",{No}
3,2008,fannie,4,{>=30% <100%},"{>0, <=80%}","{<20%, >=40%}",{prior to year aquired},{purchase},{mortgage_company},{no},{230},"{0-1, >=2}","{32.0, 1.0, 3.0, 4.0, 36.0, 6.0, 40.0, 19.0, 2...","{>60, <=80%, >50, <=60%, >=0, <=50%}",{No}
4,2008,fannie,5,{>=30% <100%},"{>0, <=80%}","{<20%, >=40%}",{prior to year aquired},{purchase},{mortgage_company},{no},{50},"{0-1, >=2}",{1.0},"{>60, <=80%, >50, <=60%, >=0, <=50%}",{No}


In [4]:
agg_cols = ['census_tract_2020', 'tract_income_ratio', 'affordability_cat', 'date_of_mortgage_note',
             'purpose_of_loan', 'type_of_seller', 'federal_guarantee', 'tot_num_units',
               'num_bedrooms', 'num_units', 'affordability_level', 'tenant_income_ind']
counts = test[agg_cols].map(len)
counts.head()

,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_bedrooms,num_units,affordability_level,tenant_income_ind
0,1,1,1,1,1,1,1,1,2,10,2,1
1,1,1,1,1,1,1,1,1,2,7,3,1
2,1,1,1,1,1,1,1,1,2,1,4,1
3,1,1,1,1,1,1,1,1,2,10,3,1
4,1,1,1,1,1,1,1,1,2,1,3,1


In [5]:
add_to_gb = []
for col in counts:
    print(f'{col} ', counts[col].unique())
    if len(counts[col].unique()) == 1:
        add_to_gb.append(col)

census_tract_2020  [1]
tract_income_ratio  [1]
affordability_cat  [1]
date_of_mortgage_note  [1]
purpose_of_loan  [1]
type_of_seller  [1]
federal_guarantee  [1]
tot_num_units  [1]
num_bedrooms  [2 1]
num_units  [10  7  1  3  8  4  5 12  6 18 16  9  2 11 14 22 13 17 19 15 39 21 20 24
 26 28 41 23 37 25 27 29 40 42 32 31 30 35 34 33]
affordability_level  [2 3 4 1 5 6]
tenant_income_ind  [1]


In [6]:
gbcols = ['year', 'enterprise_flag', 'record_number'] + add_to_gb

In [13]:
ohe_data = pd.get_dummies(mapped_data, columns=['num_bedrooms', 'affordability_level'])
ohe_data.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_units,tenant_income_ind,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
315783,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,1.0,No,False,True,False,True,False,False,False
315784,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,10.0,No,True,False,False,True,False,False,False
315785,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,1.0,No,True,False,False,True,False,False,False
315786,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,4.0,No,False,True,False,True,False,False,False
315787,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,3.0,No,True,False,False,False,False,False,True


In [29]:
ohe_data.groupby(gbcols).agg(sum).reset_index()

/tmp/ipykernel_389964/2338848404.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ohe_data.groupby(gbcols).agg(sum).reset_index()


,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,tenant_income_ind,num_units,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
0,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,No,89.0,10,7,0,13,0,0,4
1,2008,fannie,2,>=30% <100%,">0, <=80%",">=20%, <40%",prior to year aquired,refinance,mortgage_company,no,72,No,58.0,4,7,0,0,7,1,3
2,2008,fannie,3,<10%,>120%,"<20%, <40%",prior to year aquired,purchase,mortgage_company,no,264,No,7.0,4,3,2,2,2,0,1
3,2008,fannie,4,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,230,No,221.0,8,8,0,8,6,0,2
4,2008,fannie,5,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,50,No,4.0,2,2,0,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83964,2023,freddie,2591,>=30% <100%,>120%,"<20%, <40%",same year as acquired,refinance,mortgage_company,no,51-99,No,19.0,4,4,7,0,0,1,0
83965,2023,freddie,2594,>=30% <100%,">10, <=120%",">=20%, <40%",prior to year aquired,refinance,mortgage_company,no,51-99,No,32.0,8,1,0,1,4,0,4
83966,2023,freddie,2595,>=30% <100%,">0, <=80%","<20%, >=40%",same year as acquired,purchase,SAIF or BIF,no,> 149,No,188.0,5,11,0,12,0,0,4
83967,2023,freddie,2596,>=30% <100%,">0, <=80%",">=20%, <40%",prior to year aquired,purchase,mortgage_company,no,> 149,No,125.0,3,17,0,4,1,0,15
